In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import numpy as np
import collections
import random
import tensorflow as tf
import csv

In [ ]:
from google.colab import drive
drive.mount('gdrive/')
root_path = 'gdrive/My Drive/hashes'

Drive already mounted at gdrive/; to attempt to forcibly remount, call drive.mount("gdrive/", force_remount=True).


In [ ]:
class Tensorboard:
    """
    Custom Tensorboard to post experiment summaries.
    """

    def __init__(self, logdir):
        self.writer = tf.compat.v1.summary.FileWriter(logdir)

    def close(self):
        self.writer.close()

    def log_summary(self, hash_codes, coverage, computation, iteration):
        summary = tf.Summary(value=[
            tf.Summary.Value(tag='DeepHash/HashCodes',
                             simple_value=hash_codes),
            tf.Summary.Value(tag='DeepHash/Coverage',
                             simple_value=coverage),
            tf.Summary.Value(tag='DeepHash/Computation',
                             simple_value=computation)])

        self.writer.add_summary(summary, iteration)

        self.writer.flush()

In [ ]:
descriptor="_generated_hashes_codelength128_simweight_abitbetter_smaller_probs__probs_128_128"
tensorboard = Tensorboard("gdrive/My Drive/ERProject/DeepHash/"+descriptor)
slim = tf.contrib.slim
row_size=9004# size of the dimension we are limiting our embedding  
code_length=16 #6#16# can be modified depending upon computation and coverage
external_dropout=0.8 #Percentage of things that will remain
internal_dropout=0.65 #Percentage of things that will remain
optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01)
batch_size= 32
total_it=10000#0000 #Total number of iterations, each iteration has a number of batches and gives a report at the end
num_epochs=2 #Epochs per batch
report_interval=1000
similarity_weight=1.0 #This is a hyper-parameter we can use to boost the importance of the similarity in the computation
use_probs=True
location="gdrive/My Drive/ERProject/Data/"
df = pd.read_csv(location+"pagetitle_embedding_skigram_loss_0.40_200407.csv", header=0)
df2 = pd.read_csv(location+"binary_codes.csv", header=0, dtype={'300': str, '301': str})
df2.drop(['spec_id','page_title','brand','model'], 1, inplace=True)
#print(df2.head())
#df2 = pd.read_csv(location+"ProductIdsPageTitles.csv", header=0)
#df2.drop(df2.columns.difference(['spec_id']), 1, inplace=True)
#df1.drop(columns=[100], inplace=True)
df=  pd.concat([df.reset_index(drop=True), df2.reset_index(drop=True)], axis= 1)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
# Our first dataset is now stored in a Pandas Dataframe
#df[[x for x in range(300, row_size)]] = pd.DataFrame([[0 for x in range(300, row_size)]], index=df.index)
#for item in range(300,row_size):
#  df[item]=0
#  print(str(item)+"/"+str(row_size))
#Here we check the schema, and its length
#print("Schema: "+str(df.columns))
#print("Number of rows: "+str(len(df)))
print(df.head())

df=df.rename(columns={'model_binary':str(300),'brand_binary':str(301)})

print(df.head())
df.columns = df.columns.map(str)
ground_truth= pd.read_csv(location+"final_extended_groundtruth.csv", header=0)
match_pair=dict()
non_match_pair=dict()
print(df['300'])
emb_dict={}
for index, row in df.iterrows():
  emb_dict[row["spec_id"].replace("'","")]=[row[str(item)] for item in range(0,302)]
print(tuple(emb_dict.keys())[0])

         spec_id         0         1  ...       299             300         301
0  buy.net//4233  0.031221  0.006270  ... -0.006325  00111010100101  0011111000
1  buy.net//4236  0.005103  0.029665  ... -0.008783  00101111100110  0100010011
2  buy.net//4239  0.005655  0.043519  ... -0.006725  00110000001010  0000110101
3  buy.net//4247 -0.000266  0.020450  ...  0.037856  00000110000110  0000110101
4  buy.net//4255  0.016109  0.042931  ...  0.041887  01000101101011  0001010000

[5 rows x 303 columns]
         spec_id         0         1  ...       299             300         301
0  buy.net//4233  0.031221  0.006270  ... -0.006325  00111010100101  0011111000
1  buy.net//4236  0.005103  0.029665  ... -0.008783  00101111100110  0100010011
2  buy.net//4239  0.005655  0.043519  ... -0.006725  00110000001010  0000110101
3  buy.net//4247 -0.000266  0.020450  ...  0.037856  00000110000110  0000110101
4  buy.net//4255  0.016109  0.042931  ...  0.041887  01000101101011  0001010000

[5 rows x 303 c

In [ ]:
for index, row in ground_truth.iterrows():
  if row['label']==1:
    if not row['left_spec_id'] in match_pair:
      match_pair[row['left_spec_id']]=set()
    match_pair[row['left_spec_id']].add(row['right_spec_id'])
    if not row['right_spec_id'] in match_pair:
      match_pair[row['right_spec_id']]=set()
    match_pair[row['right_spec_id']].add(row['left_spec_id'])
  else:
    if not row['left_spec_id'] in non_match_pair:
      non_match_pair[row['left_spec_id']]=set()
    non_match_pair[row['left_spec_id']].add(row['right_spec_id'])
    if not row['right_spec_id'] in non_match_pair:
      non_match_pair[row['right_spec_id']]=set()
    non_match_pair[row['right_spec_id']].add(row['left_spec_id'])

In [ ]:
split_boundary=33 #Write 33 for pickign 33% of the data for testing and the remaining for training.
test_set_matches=set()
test_set_non_matches=set()
for first_item in match_pair.keys():
  for second_item in match_pair[first_item]:
    if first_item < second_item:#Our tuple formation criteria
      if random.randrange(1,101) <=split_boundary:
        test_set_matches.add((first_item,second_item))

In [ ]:
for first_item in non_match_pair.keys():
  for second_item in non_match_pair[first_item]:
    if first_item < second_item:#Our tuple formation criteria
      if random.randrange(1,101) <=split_boundary:
        test_set_non_matches.add((first_item,second_item))

In [ ]:
import math
def emb_to_array(emb):
  arr=[float(emb[x]) for x in range(0,300)]
  arr2=[0.0 for x in range(0,8192)] 
  #print(emb[300])
  val=float(str(emb[300]))
  if not math.isnan(val):
    arr2[int(str(emb[300]),2)]=1.0
  arr3=[0.0 for x in range(0,512)] 
  val=float(str(emb[301]))
  #print(str(emb[301]))
  if not math.isnan(val):
    arr3[int(str(emb[301]),2)]=1.0
  return arr+arr2+arr3

In [ ]:
def deep_hash_network(code_length, network_type, input):
    net = tf.nn.dropout(slim.fully_connected(input, 128, activation_fn=tf.nn.relu), external_dropout)
    net = tf.nn.dropout(slim.fully_connected(net, 128, activation_fn=tf.nn.relu), internal_dropout)#, weights_regularizer=slim.l2_regularizer(1e-8))
    #net = tf.nn.dropout(slim.fully_connected(net, int(code_length/2), activation_fn=tf.nn.relu), internal_dropout)#, weights_regularizer=slim.l2_regularizer(1e-8))  
    #net = slim.fully_connected(net, 32, activation_fn=tf.nn.relu)#, weights_regularizer=slim.l2_regularizer(1e-8))  
    hash_code = tf.nn.dropout(slim.fully_connected(net, code_length, activation_fn=None), 1.0)#  <- why?
    return network_type(hash_code)

In [ ]:
our_net=deep_hash_network
print("Network function defined")

tf_device='/gpu:*'
print("Device defined")
    
shape=(1,row_size) 
print("I/O shapes defined")
    
def _network_template(state):# <-Purpose?
    return our_net(code_length, collections.namedtuple('DQH_network', ['hash_values']), state)
print("Network wrapper function defined")
    
batch_outputs1=[]# match
batch_outputs2=[]# match
batch_outputs3=[]# non_match

Network function defined
Device defined
I/O shapes defined
Network wrapper function defined


In [ ]:
def _build_network():
    global batch_outputs1, batch_outputs2, batch_outputs3
    net= tf.make_template('network', _network_template)
    batch_outputs1=tf.clip_by_value(net(states1_ph),-1.,1.)#<-setting min and max value?
    batch_outputs2=tf.clip_by_value(net(states2_ph),-1.,1.)
    batch_outputs3=tf.clip_by_value(net(states3_ph),-1.,1.)      
print("Network forward pass function defined")

Network forward pass function defined


In [ ]:
#RMSPRop
def _build_train_op(): 
  #This defines our training operation, based on: Li, Wu-Jun, Sheng Wang, and Wang-Cheng Kang. 
  #"Feature learning based deep supervised hashing with pairwise labels." 
  #arXiv preprint arXiv:1511.03855 (2015). However we extend it to a triple, because it worked better for us.
  theta=tf.divide(tf.reduce_sum(tf.multiply(batch_outputs1[0],batch_outputs2[0]),1),2)
  theta2=tf.divide(tf.reduce_sum(tf.multiply(batch_outputs1[0],batch_outputs3[0]),1),2)
  theta5=tf.divide(tf.reduce_sum(tf.multiply(batch_outputs2[0],batch_outputs3[0]),1),2)
  
  sim_loss=-(
      -tf.math.log(1+tf.math.exp(theta))+similarity_weight[0]*theta 
      
  )

  disim_loss2=-(
      -tf.math.log(1+tf.math.exp(theta2))
      
  )

  disim_loss5=-(
      -tf.math.log(1+tf.math.exp(theta5))
      
  )
  loss=sim_loss+disim_loss2+disim_loss5
  print_op=tf.print(tf.reduce_mean(loss, axis=0))
  with tf.control_dependencies([]):#Write print_op in there to print loss
    gvs = optimizer.compute_gradients(loss)
    capped_gvs = [(tf.clip_by_value(grad, -10., 10.), var) for grad, var in gvs] #We clip by value to avoid exploiding gradients -10 to 10
    return optimizer.apply_gradients(capped_gvs)
print("Network backprop pass function (loss definition and minimization criteria) defined")

Network backprop pass function (loss definition and minimization criteria) defined


In [ ]:
size_of_embedding=row_size
with tf.device(tf_device):
    batch_outputs1=tf.placeholder(tf.float32, name='bo1_ph')
    batch_outputs2=tf.placeholder(tf.float32, name='bo2_ph')
    batch_outputs3=tf.placeholder(tf.float32, name='bo3_ph')
    similarity_weight=tf.placeholder(tf.float32, name='sw_ph')
    states1_ph = tf.placeholder(tf.float32, (None,size_of_embedding), name='state1_ph')  
    states2_ph = tf.placeholder(tf.float32, (None,size_of_embedding), name='state2_ph')
    states3_ph = tf.placeholder(tf.float32, (None,size_of_embedding), name='state3_ph')
    net= _build_network()
    _train_op = _build_train_op()    
print("Device selected and variables initialized")

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Device selected and variables initialized


In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
_sess = tf.Session('', config=config)
init_op = tf.initialize_all_variables()
_sess.run(init_op)
print("Tensorflow session initalized")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Tensorflow session initalized


In [ ]:
coverage=0
choices_selected=dict()#UPDATE LOGIC
for item in list(match_pair.keys()):
    choices_selected[item]=1
sumcs=0
for item in choices_selected.keys():#PUT THIS ALSO DOWN
    sumcs+=choices_selected[item]
probs=[choices_selected[key]/sumcs for key in list(match_pair.keys())]

for num in range(0,total_it):
    #print(num)
    if use_probs:
        KeysSelected = random.choices(list(match_pair.keys()), k=batch_size, weights=probs)
        weights=[]
        for a in KeysSelected:
            mplist=[choices_selected[c] for c in list(match_pair[a])]
            sum_mplist=sum(mplist)
            mplist=[c/sum_mplist for c in mplist]
            weights.append(mplist)
        MatchesSelected = [random.choices(list(match_pair[KeysSelected[a]]), k=1, weights=weights[a])[0].replace("'","") for a in range(0,len(KeysSelected))]
    else:
        KeysSelected = random.choices(list(match_pair.keys()), k=batch_size)
        MatchesSelected = [random.choice(list(match_pair[KeysSelected[a]])).replace("'","") for a in range(0,len(KeysSelected))]
    firsts=[]
    seconds=[]
    non_matches=[]
    for i in range(0,batch_size):
        small_key=None
        big_key=None
        if KeysSelected[i]<=MatchesSelected[i]:
          small_key=KeysSelected[i]
          big_key=MatchesSelected[i]
        else:
          big_key=KeysSelected[i]
          small_key=MatchesSelected[i]
        if (KeysSelected[i] in non_match_pair) and (KeysSelected[i] in emb_dict) and (MatchesSelected[i] in emb_dict) and not (small_key,big_key) in test_set_matches:
            we_selected= random.choice(tuple(non_match_pair[KeysSelected[i]])).replace("'","")
            #if not we_selected in emb_dict:
            #    we_selected="'"+we_selected+"'"
            if small_key<=we_selected:
              big_key=we_selected
            else:
              big_key=small_key
              small_key=we_selected
            if we_selected in emb_dict and not (small_key, big_key) in test_set_non_matches:    
              #print("we selected"+we_selected)
              #print(emb_dict[we_selected])
              non_matches.append(emb_to_array(emb_dict[we_selected]))
              firsts.append(emb_to_array(emb_dict[KeysSelected[i]]))
              seconds.append(emb_to_array(emb_dict[MatchesSelected[i]])) 
    #print(len(firsts))
    if (len(firsts)==0):
      print("No item in match and embeddings") 
    else:
        val=[]
        if coverage>=100:#num>total_it/4:
            val=[1.05]#+(1.0-1*(((total_it)-num)/(total_it)))]      
        elif coverage> 95:
            val=[1.05]#Was 05, 15, 18
        elif coverage> 90:
            val=[1.1]
        else:
            val=[1.1]
        for epoch in range(0,num_epochs):
            [result]=_sess.run([_train_op], feed_dict={states1_ph: np.array(firsts,dtype=np.float64), states2_ph: np.array(seconds,dtype=np.float64), states3_ph: np.array(non_matches,dtype=np.float64), similarity_weight: np.array(val, dtype=np.float64)})
        #print(num)
        if num%report_interval==999:
          test_set=[]
          keys=list(emb_dict.keys())
          for item in keys:
             test_set.append(emb_to_array(emb_dict[item]))
          b1= np.sign(_sess.run(batch_outputs1, {states1_ph: np.array(np.array(test_set,dtype=np.float64)), similarity_weight: np.array([1.0], dtype=np.float64)})[0])
          hash_code_dict={}
          key_2_hash_code_dict={}
          for item in range(0,len(keys)):
            hash_code="".join([str(int(a)) for a in b1[item]])
            hash_code=hash_code.replace("-1","0")
            if not hash_code in hash_code_dict:
              hash_code_dict[hash_code]=set()
            set_to_use=hash_code_dict[hash_code]
            set_to_use.add(keys[item].replace("'",""))
            hash_code_dict[hash_code]=set_to_use
            key_2_hash_code_dict[keys[item].replace("'","")]=hash_code
          #print(len(key_2_hash_code_dict.keys()))
          print("Iteration: "+str(num))
          print("Number of hash codes: "+str(len(hash_code_dict.keys())))
          total_matches=0
          good_matches=0
          faulty_items=set()
          now=True
          if use_probs:
            choices_selected=dict()#UPDATE LOGIC
            for item in list(match_pair.keys()):
              choices_selected[item]=1
          for k in match_pair.keys():
            if not k in key_2_hash_code_dict:
              if not k in faulty_items:
                faulty_items.add(k)
                print("Adding "+k+", from key_2_hash_code")
            else:
              kscode= key_2_hash_code_dict[k]
              for item in list(match_pair[k]):            
                if not item in key_2_hash_code_dict:
                  if not item in faulty_items:
                    faulty_items.add(item)
                else:
                  total_matches+=1
                  if kscode == key_2_hash_code_dict[item]:
                    good_matches+=1
                  elif use_probs:
                    choices_selected[item]+=1
                    choices_selected[k]+=1
          if use_probs:
            sumcs=0
            for item in choices_selected.keys():#PUT THIS ALSO DOWN
              sumcs+=choices_selected[item]
            probs=[choices_selected[key]/sumcs for key in list(match_pair.keys())]
          coverage=100*good_matches/total_matches
          print("Coverage is: "+str(coverage))
          print("Non products keys: "+str(len(faulty_items)))
          computation=0
          for key in hash_code_dict:
            if len(hash_code_dict[key])>1:
              computation+=(len(hash_code_dict[key])*len(hash_code_dict[key]))
          computation=100*computation/(len(key_2_hash_code_dict.keys())*len(key_2_hash_code_dict.keys()))
          print("Computation is "+str(computation))
          tensorboard.log_summary(
            len(hash_code_dict.keys()), coverage, computation, num)


from datetime import datetime
dateTimeObj = datetime.now().replace(second=0, microsecond=0)
f = open(location+str(dateTimeObj)+"_Codes_"+str(len(hash_code_dict.keys()))+"_Cov_"+str(coverage)+"_Comp_"+"%.2f" % computation+descriptor+".csv",'w')
for item in key_2_hash_code_dict.keys():
  f.write(item+","+key_2_hash_code_dict[item]+"\n")
f.close()

counter=0
for item in sorted(list(faulty_items)):
  print(str(counter)+"-"+item)
  counter+=1



Iteration: 999
Number of hash codes: 1994
Coverage is: 92.10125752289066
Non products keys: 0
Computation is 5.162397615436606
Iteration: 1999
Number of hash codes: 832
Coverage is: 56.323911668969394
Non products keys: 0
Computation is 8.142728000444178
Iteration: 2999
Number of hash codes: 574
Coverage is: 86.44357539282954
Non products keys: 0
Computation is 5.381768072713913
Iteration: 3999
Number of hash codes: 818
Coverage is: 81.97316347797579
Non products keys: 0
Computation is 43.2317266240862
Iteration: 5999
Number of hash codes: 1134
Coverage is: 74.47251949511744
Non products keys: 0
Computation is 23.201491282619582
Iteration: 6999
Number of hash codes: 892
Coverage is: 75.44903168395663
Non products keys: 0
Computation is 18.15939814106076
Iteration: 7999
Number of hash codes: 1224
Coverage is: 71.02077137437651
Non products keys: 0
Computation is 15.734729884100144
Iteration: 8999
Number of hash codes: 1301
Coverage is: 96.04945788352106
Non products keys: 0
Computation 